In [12]:
import Generate_stock_data as GD
import numpy as np
from __future__ import unicode_literals, print_function, division, absolute_import
from io import open
import unicodedata, string, re, random, math, time, shutil, pdb

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms

use_cuda = torch.cuda.is_available()

In [13]:
# Parameters
learning_rate = 0.001
training_iters = 50000
batch_size = 128
display_step = 100
model_path = "./stock_dual/"

# Network Parameters
# encoder parameter
n_input_encoder = 81 # n_feature of encoder input
n_steps_encoder = 10 # time steps
n_hidden_encoder = 128 # size of hidden units

# decoder parameter
n_input_decoder = 1
n_steps_decoder = 9
n_hidden_decoder = 128
n_classes = 1 # size of the decoder output

loss_value = []
step_value = []
loss_test=[]
loss_val = []

dtype = torch.FloatTensor

# Get the data from here.
Data = GD.Input_data(batch_size, n_steps_encoder, n_steps_decoder, n_hidden_encoder)

In [14]:

# class AttnDec(nn.Module):
#     def __init__(self, decoder_state_size, encoder_state_size):
#         super(AttnDec, self).__init__()
#         self.decoder_state_size = decoder_state_size
#         self.cell_state_size = decoder_state_size
#         self.encoder_state_size = encoder_state_size
#         self.Attn_Decoder_V = nn.Parameter(torch.FloatTensor(self.encoder_state_size,1))
#         self.Attn_Decoder_W = nn.Linear(self.decoder_state_size + self.cell_state_size, self.encoder_state_size)
#         self.Attn_Decoder_U = nn.Linear(self.encoder_state_size, self.encoder_state_size)

#     def forward(self, decoder_state, cell_state, encoder_outputs):
#         '''
#         decoder_state(batch_size, decoder_state_size): d_t
#         cell_state(batch_size, cell_state_size): s_t
#         encoder_outputs(batch_size, n_steps_encoder, hidden_size): h_1 h_2 ... h_T
#         '''
#         this_batch_size = encoder_outputs.size(0)
#         max_len = encoder_outputs.size(1)
#         # change w v u  linears to all variables - prahal
#         # no need for the for loop because we can do axis wise multiplication
#         # Create variable to store attention energies
#         attn_energies = Variable(torch.zeros(this_batch_size, max_len)) # B x T

#         if use_cuda:
#             attn_energies = attn_energies.cuda()

#         combined_state = torch.cat((decoder_state, cell_state),0)
#         # Calculate energy for each encoder output
#         for i in range(max_len):
#             attn_energies[:, i] = self.score(combined_state, encoder_outputs[:, i, :])
        
#         return F.softmax(attn_energies) # B x T
   
#     def score(self, combined_hidden_state, encoder_output):
#         '''
#         combined_hidden_state: [d_t;s_t]
#         encoder_output: h_t
#         '''
#         #v^T*tanh(W*[d_t;s_t] + U*h_i)
#         return F.tanh(self.Attn_Decoder_W(combined_hidden_state) + self.Attn_Decoder_U(encoder_output)).mm(self.Attn_Decoder_V)
        
        

In [15]:
class Attn(nn.Module): #Exactly same as AttnDec. It's redundant.
    def __init__(self, decoder_state_size, encoder_state_size): #(128, 10)
        super(Attn, self).__init__()
        self.decoder_state_size = decoder_state_size
        self.cell_state_size = decoder_state_size
        self.encoder_state_size = encoder_state_size
        self.Attn_Decoder_V = nn.Parameter(torch.FloatTensor(self.encoder_state_size,1))
        self.Attn_Decoder_W = nn.Linear(self.decoder_state_size + self.cell_state_size, self.encoder_state_size)
        self.Attn_Decoder_U = nn.Linear(self.encoder_state_size, self.encoder_state_size)

    def forward(self, decoder_state, cell_state, encoder_outputs):
        '''
        decoder_state(batch_size, decoder_state_size): h_t
        cell_state(batch_size, cell_state_size): s_t
        encoder_outputs(batch_size, n_input_encoder, n_steps_encoder): X
        '''
        this_batch_size = encoder_outputs.size(0)
        max_len = encoder_outputs.size(1)
        # change w v u  linears to all variables - prahal
        # no need for the for loop because we can do axis wise multiplication
        # Create variable to store attention energies
        # combined_state ?? double check
        # init inital cell state
        attn_energies = Variable(torch.zeros(this_batch_size, max_len)) # B x N

        if use_cuda:
            attn_energies = attn_energies.cuda()
        
        combined_state = torch.cat((decoder_state, cell_state),1)
        # Calculate energy for each encoder output
        for i in range(max_len):
            attn_energies[:, i] = self.score(combined_state, encoder_outputs[:, i, :])
        
        return F.softmax(attn_energies) # B x N
   
    def score(self, combined_hidden_state, encoder_output):
        '''
        combined_hidden_state: [d_t;s_t]
        encoder_output: h_t
        '''
#         pdb.set_trace()
        #v^T*tanh(W*[d_t;s_t] + U*h_i)
        return F.tanh(self.Attn_Decoder_W(combined_hidden_state) + self.Attn_Decoder_U(encoder_output)).mm(self.Attn_Decoder_V)
        
        

## Attention Decoder
[Reference](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb)

In [16]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, input_size = n_input_decoder, hidden_size = n_hidden_decoder, output_size = n_classes, 
                 n_layers = 1, dropout_p = 0, encoder_hidden_size = n_hidden_encoder):
        '''
        input_size: size of input y_i=1
        hidden_size: size of d_t=128
        output_size: n_classes=1
        n_layers: number of hidden layers=1
        encoder_hidden_size: size of h_t=128
        '''
        super(AttnDecoderRNN, self).__init__()
        
        # Define parameters
        self.decoder_input_size = input_size
        self.decoder_hidden_size = hidden_size #size of d_t
        self.decoder_output_size = output_size #size of y_t
        self.n_layers = n_layers
        self.encoder_hidden_size = encoder_hidden_size
        self.dropout_p = dropout_p
        
        # Define layers
        #self.dropout = nn.Dropout(dropout_p)
        self.linear_layer_input = nn.Linear(self.decoder_input_size + self.encoder_hidden_size, self.decoder_input_size)
        self.linear_layer_output = nn.Linear(self.decoder_hidden_size + self.encoder_hidden_size, self.decoder_hidden_size)
        self.decoder_cell = nn.LSTMCell(self.decoder_input_size, self.decoder_hidden_size)
        self.attn = Attn(self.decoder_hidden_size, self.encoder_hidden_size)
        #print (len(list(self.attn.parameters())))
        
    '''encoder_output:Not in use right now'''
    def forward(self, decoder_input, decoder_hidden, decoder_cell_state, encoder_outputs, context_vector):
        '''
        decoder_input(batch_size, input_size): y_i (batch_size x 1)
        decoder_hidden(batch_size, hidden_size): initial hidden state
        encoder_outputs(batch_size, encoder_size): h_1 ... h_T
                                    encoder_size-currently: (10 x 128)
        context_vector(batch_size, n_hidden_encoder): c_t
        '''
        #tempTest = Variable(torch.zeros(batch_size, self.decoder_input_size + self.encoder_hidden_size))
        concat_decoder_input = self.linear_layer_input(torch.cat((decoder_input, context_vector),1)) # B x 1
        
        #replace decoder_hidden by self.decoder_hidden_state?
        decoder_hidden, decoder_cell_state = self.decoder_cell(concat_decoder_input, (decoder_hidden, decoder_cell_state))
        
        attn_weights = self.attn(decoder_hidden, decoder_cell_state, encoder_outputs).unsqueeze(2).expand_as(encoder_outputs) # B x T
        context_vector = torch.sum(attn_weights * encoder_outputs, 1) # B x encoder_hidden_size
        decoder_output = self.linear_layer_output(torch.cat((decoder_hidden, context_vector),1)) # B x 1
        return decoder_hidden, decoder_cell_state, decoder_output, context_vector
    
    def initHidden(self, encoder_hidden, encoder_cell_state):
        decoder_hidden = encoder_hidden
        decoder_cell_state = encoder_cell_state
        if use_cuda:
            return (decoder_hidden.cuda(), decoder_cell_state.cuda())
        else:
            return (decoder_hidden, decoder_cell_state)

## Attention Encoder

In [17]:
class AttnEncoderRNN(nn.Module):
    def __init__(self, input_size = n_input_encoder, hidden_size = n_hidden_encoder, output_size = n_hidden_encoder, 
                 n_layers = 1, dropout_p = 0):
        '''
        input_size: size of input driving sequence x_t
        hidden_size: size of h_t=128
        output_size: size of h_t=128 ?
        n_layers: number of hidden layers=1
        '''
        super(AttnEncoderRNN, self).__init__()
        
        # Define parameters
        self.encoder_input_size = input_size
        self.encoder_hidden_size = hidden_size # size of h_t
        self.encoder_output_size = output_size # size of h_t
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        # Define layers
        #self.dropout = nn.Dropout(dropout_p)
        self.encoder_cell = nn.LSTMCell(self.encoder_input_size, self.encoder_hidden_size)
        self.attn = Attn(self.encoder_hidden_size, n_steps_encoder)
        #print (len(list(self.parameters())))
        #print (len(list(self.attn.parameters())))
        #print (len(list(self.parameters())))
        
    def forward(self, encoder_input, encoder_hidden, encoder_cell_state, network_input):
        '''
        encoder_input(batch_size, input_size): x_t (batch_size x 81)
        encoder_hidden(batch_size, hidden_size): initial hidden state
        network_input(batch_size, input_size, T): X
        '''
        #replace encoder_hidden by self.encoder_hidden_state?
        attn_weights = self.attn(encoder_hidden, encoder_cell_state, network_input) # B x N
        encoder_input_weighted = attn_weights * encoder_input # B x N
        
        encoder_hidden, encoder_cell_state = self.encoder_cell(encoder_input_weighted, (encoder_hidden, encoder_cell_state))
        
        return encoder_hidden, encoder_cell_state, encoder_input_weighted
    def initHidden(self, batch_size):
        encoder_hidden = Variable(torch.zeros(batch_size, self.encoder_hidden_size))
        encoder_cell_state = Variable(torch.zeros(batch_size, self.encoder_hidden_size))
        if use_cuda:
            return (encoder_hidden.cuda(), encoder_cell_state.cuda())
        else:
            return (encoder_hidden, encoder_cell_state)

## Main Functions

In [18]:
def save_checkpoint(state, is_best, filename='checkpoint.tar'):
    torch.save(state, model_path + filename)
    if is_best:
        shutil.copyfile(model_path + filename, model_path + 'model_best.tar')

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [19]:
def feedforward(input_dict, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    global n_hidden_encoder
    batch_sz = input_dict['encoder_attention_states'].shape[0]
    encoder_hidden, encoder_cell_state = encoder.initHidden(batch_sz)
    encoder_attention_states = Variable(torch.from_numpy(input_dict['encoder_attention_states']).type(dtype)) #numpy
    encoder_attn_size = input_dict['encoder_attention_states'].shape[2]
    encoder_attn_len = input_dict['encoder_attention_states'].shape[1]
    decoder_attn_size = input_dict['decoder_input'].shape[1]
    encoder_input = Variable(torch.from_numpy(input_dict['encoder_input']).permute(1,0,2).type(dtype))
    encoder_input = encoder_input.cuda() if use_cuda else encoder_input
    decoder_input = Variable(torch.from_numpy(input_dict['decoder_input']).permute(1,0,2).type(dtype))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    decoder_gt = Variable(torch.from_numpy(input_dict['decoder_gt']).type(dtype))
    decoder_gt = decoder_gt.cuda() if use_cuda else decoder_gt

    encoder_hiddens = Variable(torch.zeros(encoder_attn_size, batch_sz, n_hidden_encoder).type(dtype))
    encoder_hiddens = encoder_hiddens.cuda() if use_cuda else encoder_hiddens
    encoder_attn_weights = Variable(torch.zeros(encoder_attn_size, batch_sz,encoder_attn_len).type(dtype))
    encoder_attn_weights = encoder_attn_weights.cuda() if use_cuda else encoder_attn_weights
    decoder_outputs = Variable(torch.zeros(decoder_attn_size, batch_sz, n_hidden_decoder).type(dtype))
    decoder_outputs = decoder_outputs.cuda() if use_cuda else decoder_outputs

    loss = 0
    
    for ei in range(encoder_attn_size):
        encoder_hidden, encoder_cell_state, encoder_attention = encoder(
            encoder_input[ei], encoder_hidden, encoder_cell_state, encoder_attention_states)
        encoder_hiddens[ei] = encoder_hidden
        encoder_attn_weights[ei] = encoder_attention

    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden, decoder_cell_state = decoder.initHidden(encoder_hiddens[-1], encoder_cell_state) # Will it make a new copy or is it just a pointer?
    context_vector = torch.FloatTensor(batch_sz, n_hidden_encoder)

    # permute the encoder_ouputs [time_steps * batch_size * hidden_size] 
    # to [batch_size * time_steps * hidden_size]
    encoder_hiddens = encoder_hiddens.permute(1,0,2)
    for di in range(decoder_attn_size):
        decoder_hidden, decoder_cell_state, decoder_output, context_vector = decoder(
            decoder_input[di], decoder_hidden, decoder_cell_state, encoder_hiddens, context_vector)
        decoder_outputs[di,:,:] = decoder_output.data
    
    y_pred = decoder_outputs[-1].mm(input_dict['output_weight']) + (input_dict['output_bias'])
    #print (y_pred[0][0])
    loss = criterion(y_pred, decoder_gt)
    encoder_data = {'encoder_hiddens': encoder_hiddens, 'encoder_cell_state': encoder_cell_state, 'encoder_attn_weights': encoder_attn_weights};
    decoder_data = {'decoder_outputs': decoder_outputs, 'decoder_cell_state': decoder_cell_state, 'context_vector': context_vector}
    return loss, encoder_data, decoder_data


In [20]:
def train(input_dict, encoder, decoder, encoder_optimizer, decoder_optimizer, output_optimizer, criterion):
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    output_optimizer.zero_grad()
    
    loss, encoder_data, decoder_data = feedforward(input_dict, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
    
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    output_optimizer.step()

    return loss.data[0], encoder_data, decoder_data

In [21]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100):
    global learning_rate, n_classes, batch_size, n_steps_encoder, n_steps_decoder, n_hidden_encoder
    start = time.time()

    output_weight = Variable(torch.randn(n_hidden_decoder, n_classes).type(dtype), requires_grad=True)
    output_weight = output_weight.cuda() if use_cuda else output_weight
    output_bias = Variable(torch.randn(n_classes).type(dtype), requires_grad=True)
    output_bias = output_bias.cuda() if use_cuda else output_bias
    
    encoder_optimizer = optim.RMSprop(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.RMSprop(decoder.parameters(), lr=learning_rate)
    output_optimizer = optim.RMSprop([output_weight, output_bias], lr=learning_rate)

    criterion = nn.MSELoss()
    count = 1

    # read the input data
    Data = GD.Input_data(batch_size, n_steps_encoder, n_steps_decoder, n_hidden_encoder)
    
    #print (len(list(encoder.parameters())))
    #print (len(list(decoder.parameters())))
    for iter in range(1, n_iters + 1):
        # the shape of batch_x is (batch_size, n_steps, n_input)
        batch_x, batch_y, prev_y, encoder_states = Data.next_batch()
        input_dict = {'encoder_input': batch_x, 'decoder_gt': batch_y, 'decoder_input': prev_y,
            'encoder_attention_states':encoder_states, 'output_weight': output_weight, 'output_bias': output_bias}

        loss, encoder_data, decoder_data = train(input_dict, encoder, decoder, encoder_optimizer, decoder_optimizer, output_optimizer, criterion)

        if iter % print_every == 0:
            print('Time : %s ' % (timeSince(start, iter / n_iters)))
            print("Iter " + str(iter) + ", Minibatch Loss= " + "{:.6f}".format(loss))

            #store the value
            loss_value.append(loss)
            step_value.append(iter)

            # validation
            val_x, val_y, val_prev_y, encoder_states_val = Data.validation()
            val_dict = {'encoder_input': val_x, 'decoder_gt': val_y, 'decoder_input': val_prev_y,
                         'encoder_attention_states':encoder_states_val, 'output_weight': output_weight, 'output_bias': output_bias}

            loss_val1, _, _  = feedforward(val_dict, encoder, decoder,
                    encoder_optimizer, decoder_optimizer, criterion)
            loss_val1 = loss_val1.data[0]
            loss_val.append(loss_val1)
            print("validation Accuracy:", loss_val1)

            # testing
            test_x, test_y, test_prev_y, encoder_states_test= Data.testing()
            test_dict = {'encoder_input': test_x, 'decoder_gt': test_y, 'decoder_input': test_prev_y,
                         'encoder_attention_states':encoder_states_test, 'output_weight': output_weight, 'output_bias': output_bias}
            loss_test1, _, _ = feedforward(test_dict, encoder, decoder,
                    encoder_optimizer, decoder_optimizer, criterion)
            loss_test1 = loss_test1.data[0]
            loss_test.append(loss_test1)

            print("Testing Accuracy:", loss_test1)

            #save the parameters
            if loss_val1<=min(loss_val):
                is_best = True
                save_checkpoint({
                    'epoch': iter + 1,
                    'encoder_data': encoder_data,
                    'decoder_data': decoder_data,
                    'encoder_optimizer' : encoder_optimizer.state_dict(),
                    'decoder_optimizer' : decoder_optimizer.state_dict(),
                    'output_optimizer' : output_optimizer.state_dict(),
                }, is_best)

        count += 1

        # reduce the learning rate
        if count > 10000:
            learning_rate *= 0.1
            count = 0
            is_best = False
            save_checkpoint({
                'epoch': iter + 1,
                'encoder_data': encoder_data,
                'decoder_data': decoder_data,
                'encoder_optimizer' : encoder_optimizer.state_dict(),
                'decoder_optimizer' : decoder_optimizer.state_dict(),
                'output_optimizer' : output_optimizer.state_dict(),
            }, is_best)

In [22]:
# def main():
attn_encoder = AttnEncoderRNN()
attn_decoder = AttnDecoderRNN()

if use_cuda:
    attn_encoder = encoder.cuda()
    attn_decoder = attn_decoder.cuda()

trainIters(attn_encoder, attn_decoder, training_iters, print_every=display_step)
print("Optimization Finished!")

Time : 0m 53s (- 445m 54s) 
Iter 100, Minibatch Loss= nan
validation Accuracy: nan
Testing Accuracy: nan
Time : 1m 46s (- 442m 11s) 
Iter 200, Minibatch Loss= nan
validation Accuracy: nan
Testing Accuracy: nan
Time : 2m 30s (- 416m 24s) 
Iter 300, Minibatch Loss= nan
validation Accuracy: nan
Testing Accuracy: nan
Time : 3m 13s (- 400m 23s) 
Iter 400, Minibatch Loss= nan
validation Accuracy: nan
Testing Accuracy: nan
Time : 4m 2s (- 400m 41s) 
Iter 500, Minibatch Loss= nan
validation Accuracy: nan
Testing Accuracy: nan
Time : 5m 6s (- 420m 11s) 
Iter 600, Minibatch Loss= nan
validation Accuracy: nan
Testing Accuracy: nan
Time : 6m 2s (- 424m 57s) 
Iter 700, Minibatch Loss= nan
validation Accuracy: nan
Testing Accuracy: nan
Time : 6m 47s (- 417m 37s) 
Iter 800, Minibatch Loss= nan
validation Accuracy: nan
Testing Accuracy: nan
Time : 7m 31s (- 410m 31s) 
Iter 900, Minibatch Loss= nan
validation Accuracy: nan
Testing Accuracy: nan
Time : 8m 17s (- 406m 34s) 
Iter 1000, Minibatch Loss= nan

KeyboardInterrupt: 

## Debugging & Testing

In [ ]:
'''
decoder_state(batch_size, decoder_state_size): d_t
cell_state(batch_size, cell_state_size): s_t
hidden_state(batch_size, hidden_size): h_t
'''
batch_size = 128
decoder_state_size = 81
cell_state_size = 81
hidden_size = 128

In [ ]:
d_t=torch.nn.Parameter(torch.FloatTensor(batch_size, decoder_state_size))
s_t=torch.nn.Parameter(torch.FloatTensor(batch_size, cell_state_size))
h_t=torch.nn.Parameter(torch.FloatTensor(batch_size, n_steps_encoder, hidden_size))
act=nn.Tanh()
d_t.size()

In [ ]:
i = 3
combined_state = torch.cat((d_t, s_t),1)

In [ ]:
combined_state.size()

In [ ]:
encoder_output = h_t

In [ ]:
encoder_output.size()

In [ ]:
Attn_Decoder_V = nn.Parameter(torch.FloatTensor(hidden_size,1))
Attn_Decoder_W = nn.Linear(decoder_state_size + cell_state_size, hidden_size)
Attn_Decoder_U = nn.Parameter(torch.FloatTensor(hidden_size, hidden_size))
print(Attn_Decoder_V.size())
print(Attn_Decoder_U.size())

In [ ]:
Attn_Decoder_W(combined_state).size()

In [ ]:
energy = Attn_Decoder_W(combined_state) + encoder_output.mm(Attn_Decoder_U)

In [ ]:
energy = energy.mm(Attn_Decoder_V)

In [ ]:
attn_energies = Variable(torch.zeros(batch_size, n_steps_encoder)) # B x T
for d in range(batch_size):
    for i in range(n_steps_encoder):
        combined_state = torch.cat((d_t[d,:].unsqueeze(0), s_t[d,:].unsqueeze(0)),1)
        encoder_output = h_t[d, i, :]
#         pdb.set_trace()
        energy=act(Attn_Decoder_W(combined_state) + Attn_Decoder_U(encoder_output)).mm(Attn_Decoder_V).squeeze(0)
        attn_energies[d,i] = energy
#     attn_energies[:,i] = energy.mm().squeeze(0)

In [ ]:
print(h_t.size())
print(F.softmax(attn_energies).size())

In [ ]:
x=F.softmax(attn_energies)
y=h_t*x.unsqueeze(2).expand_as(h_t)
y.size()

In [ ]:
# attn_energies = Variable(torch.zeros(batch_size, n_steps_encoder)) # B x T
attn_energies.backward()

In [ ]:
y=torch.sum(y,1)

In [ ]:
z=y[:,-1].unsqueeze(1)

In [ ]:
torch.cat((y,z),1).size()

In [ ]:
torch.ones(23,10).unsqueeze(2).size()

In [ ]:
x.unsqueeze(2).expand_as(torch.randn(2,2,4))[0]